## Milestone 1- Question 5: Example Joining Datasets

The purpose of this notebook is to do an example joining on the listings_detailed and calendar dataset on a listing's ID from the Broward County data before we conduct a similar dataset join for our other 14 cities. This will help give us a checkpoint with our data to see if there are any potential issues that may occur with this join.

In [2]:
import pandas as pd
import spacy
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy.stats import mannwhitneyu
from statistics import mean
from scipy.stats import norm
import numpy as np
import scipy.stats as stats
import seaborn as sns
from matplotlib.colors import ListedColormap
from collections import Counter
import geopandas as gpd
import datetime
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter

In [3]:
# Define the directory path for each city
broward_directory_path = r'C:\Users\tiffa.TIFFANY\OneDrive\Documents\DS 5460 - Big Data Scaling\Final Project\data\usa\Broward County'

# File names
listings = 'listings.csv'
listings_detailed = 'listings_detailed.csv'
reviews = 'reviews.csv'
reviews_detailed = 'reviews_detailed.csv'
calendar = 'calendar.csv'
neighbourhoods = 'neighbourhoods.csv'
neighbourhoods_json = 'neighbourhoods.geojson'

# Full paths to read in later for each city
broward_calendar_path = broward_directory_path + '\\' + calendar
broward_listings_detailed_path = broward_directory_path + '\\' + listings_detailed
broward_listings_path = broward_directory_path + '\\' + listings
broward_reviews_path = broward_directory_path + '\\' + reviews
broward_reviews_detailed_path = broward_directory_path + '\\' + reviews_detailed
broward_neighbourhoods_path = broward_directory_path + '\\' + neighbourhoods
broward_neighbourhoods_json_path = broward_directory_path + '\\' + neighbourhoods_json

In [4]:
# Read in the files
broward_calendar_df = pd.read_csv(broward_calendar_path,  na_filter=False)
broward_listings_detailed_df = pd.read_csv(broward_listings_detailed_path,  na_filter=False)

C:\Users\tiffa.TIFFANY\AppData\Local\Temp\ipykernel_10664\3421949543.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  broward_calendar_df = pd.read_csv(broward_calendar_path,  na_filter=False)
C:\Users\tiffa.TIFFANY\AppData\Local\Temp\ipykernel_10664\3421949543.py:2: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  broward_listings_detailed_df = pd.read_csv(broward_listings_detailed_path,  na_filter=False)


In [5]:
# Do some initial data cleaning to properly conduct EDA analysis
def convert_calendar_data(df):
    # Convert price columns to numeric after removing '$' and ','
    df['price'] = pd.to_numeric(df['price'].str.replace('$', '').str.replace(',', ''), errors='coerce')
    df['adjusted_price'] = pd.to_numeric(df['adjusted_price'].str.replace('$', '').str.replace(',', ''), errors='coerce')
    
    # Convert date column to datetime and extract date part
    df['date'] = pd.to_datetime(df['date']).dt.date
    df['minimum_nights'] = pd.to_numeric(df['minimum_nights'], errors='coerce')
    df['maximum_nights'] = pd.to_numeric(df['maximum_nights'], errors='coerce')
    
    return df

def convert_listings_detailed_data(df):
    # Convert price columns to numeric after removing '$' and ','
    df['price'] = pd.to_numeric(df['price'].str.replace('$', '').str.replace(',', ''), errors='coerce')
    
    # Convert date column to datetime and extract date part
    df['host_since'] = pd.to_datetime(df['host_since']).dt.date

    columns_to_convert = [
    'host_total_listings_count',
    'bedrooms',
    'beds',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'reviews_per_month'
    ]
        
    for column in columns_to_convert:
        df[column] = pd.to_numeric(df[column], errors='coerce')

    
    return df

# Applying the conversion function to each dataframe
broward_calendar_df = convert_calendar_data(broward_calendar_df)
broward_listings_detailed_df = convert_listings_detailed_data(broward_listings_detailed_df)

C:\Users\tiffa.TIFFANY\AppData\Local\Temp\ipykernel_10664\1424709187.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = pd.to_numeric(df['price'].str.replace('$', '').str.replace(',', ''), errors='coerce')
C:\Users\tiffa.TIFFANY\AppData\Local\Temp\ipykernel_10664\1424709187.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['adjusted_price'] = pd.to_numeric(df['adjusted_price'].str.replace('$', '').str.replace(',', ''), errors='coerce')
C:\Users\tiffa.TIFFANY\AppData\Local\Temp\ipykernel_10664\1424709187.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will

In [8]:
# Rename the id field to listing_id
broward_listings_detailed_df.rename(columns={'id': 'listing_id'}, inplace=True)

In [9]:
broward_calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,69824,2023-03-28,f,150.0,150.0,30.0,365.0
1,69824,2023-03-29,f,150.0,150.0,30.0,365.0
2,69824,2023-03-30,f,150.0,150.0,30.0,365.0
3,69824,2023-03-31,t,150.0,150.0,30.0,365.0
4,69824,2023-04-01,t,150.0,150.0,30.0,365.0


In [10]:
broward_listings_detailed_df.head()

,listing_id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,827736378366911479,https://www.airbnb.com/rooms/827736378366911479,20230327145536,2023-03-27,city scrape,Legion 1BR/1BA,Take it easy at this unique and tranquil getaway.,,https://a0.muscache.com/pictures/miso/Hosting-...,475630606,...,NaN,NaN,NaN,,t,1,1,0,0,NaN
1,592589963829194972,https://www.airbnb.com/rooms/592589963829194972,20230327145536,2023-03-28,city scrape,Club Wyndham Royal Vista,"Located directly on the beach, the property si...",,https://a0.muscache.com/pictures/8a429c15-ea50...,66506549,...,NaN,NaN,NaN,,f,5,0,5,0,NaN
2,772438920837360569,https://www.airbnb.com/rooms/772438920837360569,20230327145536,2023-03-27,city scrape,Relaxing 5 Acre Ranch home with private pond!,Relax with the whole family at this peaceful p...,,https://a0.muscache.com/pictures/miso/Hosting-...,382318476,...,5.0,5.00,5.00,,f,1,1,0,0,0.65
3,33271346,https://www.airbnb.com/rooms/33271346,20230327145536,2023-03-28,city scrape,Beach Escape – One Block from the Beach!,Newly constructed and beautifully renovated Ke...,Pompano world-famous coastline is a very popul...,https://a0.muscache.com/pictures/0f2305d2-7400...,118856968,...,4.8,4.88,4.68,,t,3,3,0,0,2.69
4,484515,https://www.airbnb.com/rooms/484515,20230327145536,2023-03-28,city scrape,MIAMI- AMAZING APARTMENT OVER BEACH,<b>The space</b><br />The apartment is located...,,https://a0.muscache.com/pictures/6791581/6de56...,637272,...,4.6,4.64,4.44,,f,6,6,0,0,0.21


In [19]:
# Pivot the broward_calendar_df to get dates as columns and prices as values.
calendar_pivot = broward_calendar_df.pivot(index='listing_id', columns='date', values='price')

# Create new column names with 'date-price (MM.DD.YY)' format
calendar_pivot.columns = calendar_pivot.columns.to_series().apply(lambda x: f'date-price ({x.month}.{x.day}.{x.year})')

calendar_pivot.reset_index(inplace=True)

mega_table_transposed = pd.merge(broward_listings_detailed_df, calendar_pivot, on='listing_id', how='left')

mega_table_transposed.head()

,listing_id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,date-price (3.22.2024),date-price (3.23.2024),date-price (3.24.2024),date-price (3.25.2024),date-price (3.26.2024),date-price (3.27.2024),date-price (3.28.2024),date-price (3.29.2024),date-price (3.30.2024),date-price (3.31.2024)
0,827736378366911479,https://www.airbnb.com/rooms/827736378366911479,20230327145536,2023-03-27,city scrape,Legion 1BR/1BA,Take it easy at this unique and tranquil getaway.,,https://a0.muscache.com/pictures/miso/Hosting-...,475630606,...,222.0,222.0,222.0,222.0,NaN,NaN,NaN,NaN,NaN,NaN
1,592589963829194972,https://www.airbnb.com/rooms/592589963829194972,20230327145536,2023-03-28,city scrape,Club Wyndham Royal Vista,"Located directly on the beach, the property si...",,https://a0.muscache.com/pictures/8a429c15-ea50...,66506549,...,500.0,500.0,500.0,500.0,500.0,NaN,NaN,NaN,NaN,NaN
2,772438920837360569,https://www.airbnb.com/rooms/772438920837360569,20230327145536,2023-03-27,city scrape,Relaxing 5 Acre Ranch home with private pond!,Relax with the whole family at this peaceful p...,,https://a0.muscache.com/pictures/miso/Hosting-...,382318476,...,500.0,500.0,500.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN
3,33271346,https://www.airbnb.com/rooms/33271346,20230327145536,2023-03-28,city scrape,Beach Escape – One Block from the Beach!,Newly constructed and beautifully renovated Ke...,Pompano world-famous coastline is a very popul...,https://a0.muscache.com/pictures/0f2305d2-7400...,118856968,...,249.0,299.0,299.0,249.0,249.0,NaN,NaN,NaN,NaN,NaN
4,484515,https://www.airbnb.com/rooms/484515,20230327145536,2023-03-28,city scrape,MIAMI- AMAZING APARTMENT OVER BEACH,<b>The space</b><br />The apartment is located...,,https://a0.muscache.com/pictures/6791581/6de56...,637272,...,330.0,330.0,330.0,330.0,330.0,NaN,NaN,NaN,NaN,NaN


##### Get standard EDA for the joined such as number of rows & columns and missing values

In [22]:
# Calculate total rows and columns
total_rows = mega_table_transposed.shape[0]
total_columns = mega_table_transposed.shape[1]
print(f"Total Rows: {total_rows}, Total Columns: {total_columns}")

# Missing values
missing_values_count = mega_table_transposed.isnull().sum()
total_missing = missing_values_count.sum()
print(f"Total Missing Values: {total_missing}")
if total_missing > 0:
    print("Missing Values by Column:")
    for column, missing_count in missing_values_count.iteritems():
        if missing_count > 0:
            print(f" - {column}: {missing_count} missing values")

Total Rows: 16899, Total Columns: 446
Total Missing Values: 126556
Missing Values by Column:
 - host_since: 1 missing values
 - host_total_listings_count: 1 missing values
 - bedrooms: 1332 missing values
 - beds: 197 missing values
 - review_scores_rating: 2837 missing values
 - review_scores_accuracy: 2876 missing values
 - review_scores_cleanliness: 2876 missing values
 - review_scores_checkin: 2876 missing values
 - review_scores_communication: 2876 missing values
 - review_scores_location: 2876 missing values
 - review_scores_value: 2876 missing values
 - reviews_per_month: 2837 missing values
 - date-price (3.27.2023): 11321 missing values
 - date-price (3.28.2023): 4 missing values
 - date-price (3.29.2023): 4 missing values
 - date-price (3.30.2023): 4 missing values
 - date-price (3.31.2023): 4 missing values
 - date-price (4.1.2023): 4 missing values
 - date-price (4.2.2023): 2 missing values
 - date-price (4.3.2023): 2 missing values
 - date-price (4.4.2023): 2 missing value

In [21]:
# Save a copy of the joined data
# mega_table_transposed.to_csv("Example-Join-Broward_County.csv", index=False)

# print("DataFrame has been successfully written to CSV file.")

DataFrame has been successfully written to CSV file.
